In [1]:
import torch
from torch.utils.data import DataLoader

import numpy as np

from rpn.build_rpn import RPN_Model
from sam.build_sam import SAM_Model
from transformers import pipeline

from data_builder.build_dataset import PlanetscopeDataset

import os
import time
import json

from eval_utils import filter_boxes_predictions, filter_boxes, calculate_iou, calculate_precision_recall, filter_masks
from eval_utils import generate_random_bounding_boxes, generate_uniform_spaced_bounding_boxes

import pandas as pd
from rpn import _transforms as T
from torch.utils.data import ConcatDataset

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_transform(image_enhancement="FALSE"):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    if image_enhancement == "TRUE":
        transforms.append(T.ContrastBasedAdaptiveGammaCorrection())
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

#### Read and Print the evaluation config from the JSON file

In [3]:
#Create a folder with unix timestamp
root_path = os.getcwd() + '/results/' + str(int(time.time())) + '/'
os.mkdir(root_path)

with open('eval_config.json', 'r') as f:
    eval_config = json.load(f)


print("Evaluation Configuration")
print(json.dumps(eval_config, indent=1))

Evaluation Configuration
{
 "DATASET_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_rwanda/all_dataset/",
 "DATASET": "RWANDA",
 "MODEL": "SAM",
 "IMAGE_ENHANCEMENT": "FALSE",
 "FILTRATION": "FALSE",
 "TRAIN_TYPE": "FINETUNE",
 "NMS_THRESHOLD": 0.9,
 "PRED_CONFIDENCE_THRESHOLD": 0.6,
 "ENSEMBLE_BOX_OVERLAP_THRESHOLD": 0.5,
 "ENSEMBLE_BOX_BETA": 0.4,
 "RPN_MODEL_PATH": "rpn/checkpoints/1692043128/rpn_model_1.09.pth",
 "SAM_MODEL_PATH": "sam/checkpoint/sam_vit_l_0b3195.pth"
}


#### Set the device and load the dataset for evaluation

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset_test = PlanetscopeDataset(eval_config['DATASET_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))
# dataset2_test = PlanetscopeDataset(eval_config['DATASET2_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))

# dataset_test = ConcatDataset([dataset1_test, dataset2_test])

torch.manual_seed(1)
indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[-int(len(indices)*0.2):])

#### Load the proper model to be used for evaluation

In [5]:
if eval_config['MODEL'] == 'SAT-SAM':
    rpn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE'])
    sam_model = SAM_Model(eval_config['SAM_MODEL_PATH'], 'large', device)
elif eval_config['MODEL'] == 'SAM':
    vanilla_sam_model = pipeline("mask-generation", model="facebook/sam-vit-large", device=device)
elif eval_config['MODEL'] == 'MASKRCNN':
    maskrcnn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE']) #Load the Pre-Trained MaskRCNN model

In [6]:
def pixel_accuracy(predicted, target):
    correct_pixels = np.sum(predicted == target)
    total_pixels = predicted.shape[0] * predicted.shape[1]
    accuracy = correct_pixels / total_pixels
    return accuracy

def f1_score(predicted, target):
    true_positive = np.sum(np.logical_and(predicted == 1, target == 1))
    false_positive = np.sum(np.logical_and(predicted == 1, target == 0))
    false_negative = np.sum(np.logical_and(predicted == 0, target == 1))
    
    precision = true_positive / (true_positive + false_positive + 1e-8)
    recall = true_positive / (true_positive + false_negative + 1e-8)
    
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

def mean_iou(predicted, target):
    intersection = np.logical_and(predicted, target)
    union = np.logical_or(predicted, target)
    
    intersection_area = np.sum(intersection)
    union_area = np.sum(union)
    
    iou = intersection_area / (union_area + 1e-8)
    return iou

def precision_at_iou_threshold(predicted, target, iou_threshold):
    true_positive = np.sum(np.logical_and(predicted == 1, target == 1))
    false_positive = np.sum(np.logical_and(predicted == 1, target == 0))
    
    iou = true_positive / (true_positive + false_positive + 1e-8)
    
    precision = 0.0
    if iou >= iou_threshold:
        precision = iou
        
    return precision

def collate_mask(masks, img_size):
    cumulative_mask = np.zeros(img_size, dtype=np.uint8)
    
    for mask in masks:
        cumulative_mask[mask > 0] = 1
    
    return cumulative_mask

In [7]:
results = pd.DataFrame(columns=['parcel_id', 'parcel_path', 'gt_mask_ct', 'pred_mask_ct', 'pixel_acc', 'F1_score', 'mIoU'])
    
for i, (id, sam_image, rpn_image, target, ensemble, path)  in enumerate(dataset_test): 
    try:
        if eval_config['MODEL'] == 'SAT-SAM':
            rpn_image = rpn_image.squeeze(0).to(device)  
            predictions = rpn_model.predict(rpn_image)
            predictions = rpn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            
            if predictions['boxes'].shape[0] == 0:
                print("No boxes detected, generating uniform boxes")
                bboxes = generate_uniform_spaced_bounding_boxes(sam_image.size[0], sam_image.size[1], int(sam_image.size[0]*0.25))
                if eval_config['FILTRATION'] == 'TRUE':
                    filtered_boxes = filter_boxes(bboxes, ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
                    low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_boxes)
                else:
                    low_res_masks, iou_predictions = sam_model.predict(sam_image, bboxes)
            else:
                if eval_config['FILTRATION'] == 'TRUE':
                    filtered_predictions = filter_boxes_predictions(predictions, ensemble, eval_config['ENSEMBLE_BOX_BETA'], eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
                    low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_predictions)
                else:
                    low_res_masks, iou_predictions = sam_model.predict(sam_image, predictions['boxes'])
            
            high_res_masks = sam_model.postprocess(low_res_masks, tuple(sam_image.size))
            pred_masks = high_res_masks.squeeze().cpu().numpy()
            
        elif eval_config['MODEL'] == 'SAM':
            print('SAM')
            outputs = vanilla_sam_model(sam_image, points_per_batch=32)
            if eval_config['FILTRATION'] == 'TRUE':
                pred_masks = filter_masks(outputs['masks'], ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            else:
                pred_masks = outputs["masks"]

        elif eval_config['MODEL'] == 'MASKRCNN':
            print('MASKRCNN')
            rpn_image = rpn_image.squeeze(0).to(device)
            predictions = maskrcnn_model.predict(rpn_image)
            predictions = maskrcnn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            if eval_config['FILTRATION'] == 'TRUE':
                pred_masks = filter_masks(np.squeeze(np.array(predictions['masks'])), ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            else:
                pred_masks = predictions['masks'].squeeze()
            pred_masks = np.array(pred_masks) > 0.5



        # Stack the target masks to get a single mask
        pred_masks = collate_mask(pred_masks, sam_image.size)
        target_masks = collate_mask(target['masks'], sam_image.size)

        pix_acc = pixel_accuracy(pred_masks, target_masks)
        f1_sc = f1_score(pred_masks, target_masks) 
        mIoU = mean_iou(pred_masks, target_masks)
        # precision_95 = precision_at_iou_threshold(pred_masks, target_masks, 0.95)

        print("Image Id: ", id)
        print("Pixel Accuracy: ", pix_acc)
        print("F1 Score: ", f1_sc)
        print("mIoU: ", mIoU)
        # print("Precision @ 0.95: ", precision_95)

        results.loc[i] = [id, path, len(target['masks']), len(pred_masks), pix_acc, f1_sc, mIoU]
    except:
        print("Error in image: ", id)
        continue

SAM
Image Id:  42
Pixel Accuracy:  0.7205088089923469
F1 Score:  0.4335268829950461
mIoU:  0.27675348117582815
SAM
Image Id:  14
Pixel Accuracy:  0.49037388392857145
F1 Score:  0.2963401160787921
mIoU:  0.1739432411041516
SAM
Image Id:  30
Pixel Accuracy:  0.5633320711096939
F1 Score:  0.5607077469307503
mIoU:  0.3895718554324009
SAM
Error in image:  18
SAM
Image Id:  67
Pixel Accuracy:  0.21432557397959184
F1 Score:  0.326913552211735
mIoU:  0.19539549550467922
SAM
Image Id:  71
Pixel Accuracy:  0.5151815609056123
F1 Score:  0.14296660819850787
mIoU:  0.07698655865528908
SAM
Image Id:  68
Pixel Accuracy:  0.35691366390306123
F1 Score:  0.23483240156174803
mIoU:  0.1330368897605972
SAM
Image Id:  27
Pixel Accuracy:  0.6736138791454082
F1 Score:  0.0858765537497664
mIoU:  0.04486469147322337
SAM
Image Id:  60
Pixel Accuracy:  0.20720065369897958
F1 Score:  0.2905133026914527
mIoU:  0.16994183468529853
SAM
Image Id:  51
Pixel Accuracy:  0.3774264588647959
F1 Score:  0.26044496072963697
m

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Image Id:  29
Pixel Accuracy:  0.29853914221938777
F1 Score:  0.4578105347845886
mIoU:  0.2968575195780577
SAM


/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Image Id:  56
Pixel Accuracy:  0.35316685267857145
F1 Score:  0.03879699592288187
mIoU:  0.019782244303165174
SAM


...

Image Id:  87
Pixel Accuracy:  0.33530970982142855
F1 Score:  0.16567018262771485
mIoU:  0.09031646562245806
SAM


/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Image Id:  65
Pixel Accuracy:  0.319271165497449
F1 Score:  0.4836018755824528
mIoU:  0.31891485002416164
SAM


/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Image Id:  53
Pixel Accuracy:  0.4210628587372449
F1 Score:  0.21990076750506538
mIoU:  0.12353287270312557


In [8]:
eval_config['MEAN_IOU'] = results['mIoU'].mean()
eval_config['MEAN_PIXEL_ACCURACY'] = results['pixel_acc'].mean()
eval_config['MEAN_F1_SCORE'] = results['F1_score'].mean()
# eval_config['MEAN_PRECISION@0.95'] = results['precision@0.95'].mean()

In [9]:
results.to_csv(root_path + 'results', index=False)

# save model and configuration
with open(root_path + 'eval_config.json', 'w') as f:
    json.dump(eval_config, f, indent=1)
    f.close()
print("Evaluation Configuration saved to " + root_path + 'train_config.json') 

Evaluation Configuration saved to /home/aghosh57/Kerner-Lab/SAT-SAM/results/1692694771/train_config.json
